In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:

from huggingface_hub import notebook_login

notebook_login()


In [3]:
from datasets import load_dataset
dataset = load_dataset("islam23/ec-news-qa-dolly_test" , split="train")
dataset = dataset.select(range(2000, 4000))
dataset = dataset.remove_columns(["answer"])
dataset

Dataset({
    features: ['question', 'context'],
    num_rows: 2000
})

In [4]:
from semantic_text_splitter import TiktokenTextSplitter

def split_to_chunks(row):
    document = row["context"]
    chunked_articles = []

    max_tokens = 512 
    splitter = TiktokenTextSplitter("gpt-3.5-turbo", trim_chunks=False)
    chunks = splitter.chunks(document, max_tokens)
    for chunk in chunks:
        chunked_articles.append(chunk)
    row["chunks"] = chunked_articles
    row["num_chunks"] = len(chunked_articles)
    return row
dataset = dataset.map(split_to_chunks) 
dataset 

c:\Users\islam\AppData\Local\Programs\Python\Python312\Lib\site-packages\dill\_dill.py:414: PicklingWarning: Cannot locate reference to <class 'builtins.TiktokenTextSplitter'>.
  StockPickler.save(self, obj, save_persistent_id)
c:\Users\islam\AppData\Local\Programs\Python\Python312\Lib\site-packages\dill\_dill.py:414: PicklingWarning: Cannot pickle <class 'builtins.TiktokenTextSplitter'>: builtins.TiktokenTextSplitter has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)
Parameter 'function'=<function split_to_chunks at 0x00000224960EBD80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warnin

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'context', 'chunks', 'num_chunks'],
    num_rows: 2000
})

In [23]:
dataset_chunked = pd.DataFrame(columns=["context"])  # Create an empty DataFrame with one column

# Iterate over each entry in the dataset and extract chunks
for entry in dataset:
    chunks = entry["chunks"]
    for chunk in chunks:
        new_row = {"context": chunk}
        dataset_chunked = pd.concat([dataset_chunked, pd.DataFrame([new_row])], ignore_index=True)

dataset_chunked

,context
0,hole initiative and so it is an extension of o...
1,Your next question comes from the line of Baty...
2,econd wave of 5G devices launching in late 201...
3,Additionally we saw improved QCT gross margins...
4,"for starters, what 5G has multiple near-term ..."
...,...
3780,"Okay. Yeah. So, our mid point for the gross ma..."
3781,hical coverage. The transition from 4G to 5G i...
3782,The exact timetable might be impacted by COVID...
3783,"n't break out our mix in detail, but you're co..."


In [57]:
from datasets import Dataset 
dataset_chunked = Dataset.from_pandas(dataset_chunked)
dataset_chunked

Dataset({
    features: ['context'],
    num_rows: 3785
})

In [60]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def _mean_pooling(model_output, attention_mask):
  '''
  This function takes the model output and attention mask and returns the mean pooled embeddings
  '''
  token_embeddings = model_output.last_hidden_state # First element of model_output contains all token embeddings
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()# Create an attention mask for padding tokens
  sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) # Sum the embeddings of the tokens over the sequence
  sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9) # Normalize the embeddings
  return sum_embeddings / sum_mask # Return the mean pooled embeddings

def embed_text(example):
    '''
    this function takes a query text and returns the embeddings
    '''
    inputs = tokenizer(
        example["context"], padding=True, truncation=True, return_tensors="pt"
    )
    with torch.no_grad():
        model_output = model(**inputs)# Pass the tokenized query text to the model
    pooled_embeds = _mean_pooling(model_output, inputs["attention_mask"])# Get the mean pooled embeddings
    example["embedding"] = pooled_embeds.cpu().numpy()
    return  example # Return the embeddings
dataset_pr = dataset_chunked.map(embed_text , batched=True, batch_size=128 )

dataset_pr 

Map:   0%|          | 0/3785 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'embedding'],
    num_rows: 3785
})

# **Upload to Qdrant**

In [62]:
# split the dataset in 4 parts
part1 = dataset_pr.select(range(0, 500))
part2 = dataset_pr.select(range(500, 1000))
part3 = dataset_pr.select(range(1000, 1500))
part4 = dataset_pr.select(range(1500, 2000))
part5 = dataset_pr.select(range(2000, 2500))
part6 = dataset_pr.select(range(2500, 3000))
part7 = dataset_pr.select(range(3000, 3500))
part8 = dataset_pr.select(range(3500, 3785))


In [76]:
import uuid
from qdrant_client.http import models
from qdrant_client import QdrantClient

class QdrantU:
        def __init__(self, collection_name):
            self.client = QdrantClient(
                url="https://5c32ac64-b1f7-4665-91eb-e321a98c02f6.europe-west3-0.gcp.cloud.qdrant.io:6333",
                api_key="Wd_RTregmznFMCyDLagJHM_7a5TjJJuFLVTuMgfjQD44-BHLnhYbUg",
            )
            self.collection_name = collection_name

        def _upload_documents_to_Qdrant(self, data):
            points = []
            for   content , embedding in zip( data["context"] , data["embedding"]):
                new_id = str(uuid.uuid4())  # Generate a new UUID for each document
                point = models.PointStruct(
                    id=new_id,
                    vector=embedding,
                    payload={
                        "chunk": content,
                    }
                )
                points.append(point)

            self.client.upsert(
                collection_name=self.collection_name,
                points=points
            )

            print("Uploaded:", len(data["embedding"]), "documents to the Qdrant database")

        def upload_to_Qdrant(self, data, batch_size=35):
            for i in range(0, len(data), batch_size):
                batch = data[i:i + batch_size]
                self._upload_documents_to_Qdrant(batch )
                print(f"Uploaded {i + len(batch)} documents")
        def close(self):
            self.client.close()
 

        
collection_name = "Evaluation_support"
# client = QdrantClient(
#                 url="https://5c32ac64-b1f7-4665-91eb-e321a98c02f6.europe-west3-0.gcp.cloud.qdrant.io:6333",
#                 api_key="Wd_RTregmznFMCyDLagJHM_7a5TjJJuFLVTuMgfjQD44-BHLnhYbUg",
#             )
# client.create_collection(
#     collection_name= collection_name,
#     vectors_config=models.VectorParams(
#         size=768,
#         distance=models.Distance.COSINE,
#     ),
# )
client = QdrantU(collection_name)
client.upload_to_Qdrant(part8)
client.close()

Uploaded: 35 documents to the Qdrant database
Uploaded 2 documents
Uploaded: 35 documents to the Qdrant database
Uploaded 37 documents
Uploaded: 35 documents to the Qdrant database
Uploaded 72 documents
Uploaded: 35 documents to the Qdrant database
Uploaded 107 documents
Uploaded: 35 documents to the Qdrant database
Uploaded 142 documents
Uploaded: 35 documents to the Qdrant database
Uploaded 177 documents
Uploaded: 35 documents to the Qdrant database
Uploaded 212 documents
Uploaded: 35 documents to the Qdrant database
Uploaded 247 documents
Uploaded: 5 documents to the Qdrant database
Uploaded 282 documents


In [ ]:
dataset = load_dataset('rachid16/Retrival_evaluation_dataset', split = 'train')
dataset = dataset.select(range(750))
df = dataset.to_pandas()
df.head()

In [ ]:
def get_docs_by_indices(docs, indices):
    """
    Retrieve document contexts from a list of indexed documents based on provided indices.

    Args:
    - docs (list): List of documents.
    - indices (list): List of indices corresponding to the desired documents.

    Returns:
    - list: List of document contexts corresponding to the provided indices.
    """
    return [docs[index] for index in indices]

In [ ]:
def identify_relevant_documents(documents, chunks):
  """
  This function identifies relevant documents based on chunks.

  Args:
      documents: A list of strings representing documents.
      chunks: A list of strings representing document chunks.

  Returns:
      A list of 0s and 1s, where 1 indicates a relevant document and 0 indicates an irrelevant document.
  """
  # Initialize an empty list to store relevance scores (0 or 1).
  relevance_scores = []

  for document in documents:
    # Check if any chunk of the document exists in the chunk_indices dictionary.
    is_relevant = any(chunk in document for chunk in chunks)

    # Assign 1 for relevant document and 0 for irrelevant document.
    relevance_scores.append(1 if is_relevant else 0)

  return relevance_scores

In [ ]:
def embed_query(query_text):
  '''
  this function takes a query text and returns the embeddings
  '''
  query_inputs = tokenizer(
      query_text, # Tokenize the query text
      padding=True, # Pad the query text to the maximum length
      truncation=True, # Truncate the query text to the maximum length if it exceeds the maximum length
      return_tensors="pt" # Return the tokenized query text as PyTorch tensors
  )

  with torch.no_grad():
      query_model_output = model(**query_inputs) # Pass the tokenized query text to the model

  query_embedding = _mean_pooling(query_model_output, query_inputs["attention_mask"]) # Get the mean pooled embeddings

  return query_embedding # Return the embeddings

In [ ]:
def search(query, limit=10):
    '''
      This function retrieves the vectors from the Qdrant database based on the query by similarity search
    '''
    query_vector = embed_query(query_text=query)
    query_result = client.search(
        collection_name=collection_name,
        query_vector=query_vector[0].tolist(),
        limit=limit,
        with_payload=True
    )
    return query_result

In [ ]:
# for each row in the dataframe, identify relevant documents based on chunks
import cohere

def store_identify_relevant_documents(row):
    query = row['question']
    search_results = search(query, limit=50)
    docs = list(set([result.payload['chunk'] for result in search_results]))
    apiKey = 'Uk9ecA7NyRDjlkow5TjszJai5cQeJSTakfTQG0CT'
    co = cohere.Client(apiKey)
    rerank_docs = co.rerank(
        query=query, documents=docs, top_n=row['num_chunks'], model="rerank-english-v3.0"
    )
    indices = [result.index for result in rerank_docs.results]
    documents = get_docs_by_indices(docs, indices)
    relevent_docs = identify_relevant_documents(documents, row['chunks'])
    row['relvent_doc'] = relevent_docs
    return row

In [ ]:
dataset = dataset.map(store_identify_relevant_documents)

In [ ]:
dataset.push_to_hub("Retrival_evaluation_dataset")